<a href="https://colab.research.google.com/github/manjunathsirur1994/DataCleaning-EDA/blob/main/Pipelines_and_composite_estimators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipelines and composite estimators

## Pipeline chaining estimators

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [34]:
# defining a pipeline
estimator = [('reduce_dim',PCA()), ('clf',SVC())]
pipe = Pipeline(estimator)
pipe

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

In [35]:
# Lets look at the stages of pipeline

pipe[0]

PCA()

In [36]:
pipe[1]

SVC()

## Tracking feature names in pipeline

In [37]:
# how to look at featuers with pipeline

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
import pandas as pd

In [38]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [39]:
x = df.drop(columns='target')
y = df['target']

In [40]:
estimator = [('select_k_best_parameters',SelectKBest()),('logistic_regression',LogisticRegression())]
pipe = Pipeline(estimator)
pipe

Pipeline(steps=[('select_k_best_parameters', SelectKBest()),
                ('logistic_regression', LogisticRegression())])

In [41]:
pipe.fit(x, y)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=10 is greater than n_features=4. All the features will be returned.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('select_k_best_parameters', SelectKBest()),
                ('logistic_regression', LogisticRegression())])

In [42]:
# Navigate to select_k_best_parameters and .get_feature_names_out()

print(f'lets see whats in pipe[:-1]: {pipe[:-1]}')
print(pipe[:-1].get_feature_names_out())

lets see whats in pipe[:-1]: Pipeline(steps=[('select_k_best_parameters', SelectKBest())])
['sepal length (cm)' 'sepal width (cm)' 'petal length (cm)'
 'petal width (cm)']


In [43]:
# using grid search cv in pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import GridSearchCV

param_grid = dict(reduce_dim=['passthrough', PCA(n_components=2), PCA(n_components=5), PCA(n_components=10)],
                  clf = [SVC(), LogisticRegression()],
                  clf__C=[1, 10, 20, 50, 100])

grid_search = GridSearchCV(pipe, param_grid=param_grid)

### Lets implement pipeline with grid search cv end to end [along with scalers and dimensionality reduction]

standard scaler

pca with n_components

classifers: logistic regression vs support vector classifier

In [56]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [57]:
x,y = load_digits(return_X_y=True)

In [58]:
x

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [59]:
y

array([0, 1, 2, ..., 8, 9, 8])

In [62]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [73]:
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('dim_reduction','passthrough'),
    ('classifiers','passthrough')
])

param_grid = [
    {
        'dim_reduction':[PCA()],
        'dim_reduction__n_components':[2,5,8,10],
        'classifiers':[SVC()],
        'classifiers__C':[0.1,1,10],
        'classifiers__kernel':['linear','rbf']
    },
    {
        'dim_reduction':[NMF()],
        'dim_reduction__n_components':[2,5,8,10],
        'classifiers':[SVC()],
        'classifiers__C':[0.1,1,10],
        'classifiers__kernel':['linear','rbf']
    },
    {
        'dim_reduction':[PCA()],
        'dim_reduction__n_components':[2,5,8,10],
        'classifiers':[LogisticRegression(max_iter=1000)],
        'classifiers__C':[0.01,0.1,1,10]
    },
    {
        'dim_reduction':[NMF()],
        'dim_reduction__n_components':[2,5,8,10],
        'classifiers':[LogisticRegression()],
        'classifiers__C':[0.01,0.1,1,10]
    }
]

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=10, scoring='accuracy', n_jobs=-1)
grid_search.fit(x,y)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
400 fits failed out of a total of 800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
400 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
  File "/usr/local/lib/python3.10/dist-packages/

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaling', StandardScaler()),
                                       ('dim_reduction', 'passthrough'),
                                       ('classifiers', 'passthrough')]),
             n_jobs=-1,
             param_grid=[{'classifiers': [SVC()],
                          'classifiers__C': [0.1, 1, 10],
                          'classifiers__kernel': ['linear', 'rbf'],
                          'dim_reduction': [PCA()],
                          'dim_reduction__n_components': [2, 5, 8, 10]},
                         {'classifiers': [SVC()],
                          'classifier...
                          'dim_reduction': [NMF()],
                          'dim_reduction__n_components': [2, 5, 8, 10]},
                         {'classifiers': [LogisticRegression(max_iter=1000)],
                          'classifiers__C': [0.01, 0.1, 1, 10],
                          'dim_reduction': [PCA()],
                          'dim_reduction__n_components': [2, 5, 8, 10]},
                         {'classifiers': [LogisticRegression()],
                          'classifiers__C': [0.01, 0.1, 1, 10],
                          'dim_reduction': [NMF()],
                          'dim_reduction__n_components': [2, 5, 8, 10]}],
             scoring='accuracy')

In [74]:
# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Best Parameters: {'classifiers': SVC(), 'classifiers__C': 10, 'classifiers__kernel': 'rbf', 'dim_reduction': PCA(), 'dim_reduction__n_components': 10}
Best Accuracy: 0.9326815642458101


In [75]:
pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('dim_reduction',PCA(n_components=10)),
    ('classifier',SVC(C=10, kernel='rbf'))
])
pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('dim_reduction', PCA(n_components=10)),
                ('classifier', SVC(C=10))])

In [76]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('dim_reduction', PCA(n_components=10)),
                ('classifier', SVC(C=10))])

In [77]:
y_pred = pipe.predict(x_test)

In [78]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9555555555555556